In [1]:
# import necessary library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from math import log
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
pd.set_option('display.max_columns',90)

In [2]:
# Load Data

def load_data(path):
    account_train=pd.read_csv("D:/Projects/Tookitaki/test data/raw_account_70_new.csv")
    account_test=pd.read_csv("D:/Projects/Tookitaki/test data/raw_account_30_new.csv")
    data_train=pd.read_csv("D:/Projects/Tookitaki/test data/raw_data_70_new.csv",index_col="customer_no")
    data_test=pd.read_csv("D:/Projects/Tookitaki/test data/raw_data_30_new.csv",index_col="customer_no")
    enquiry_train=pd.read_csv("D:/Projects/Tookitaki/test data/raw_enquiry_70_new.csv")
    enquiry_test=pd.read_csv("D:/Projects/Tookitaki/test data/raw_enquiry_30_new.csv")
    
    return account_train,account_test,data_train,data_test,enquiry_train,enquiry_test

path='D:/Projects/Tookitaki/test data/'
account_train,account_test,data_train,data_test,enquiry_train,enquiry_test=load_data(path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (12,20,63) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
account_train.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
0,10-Nov-15,12265,20-Oct-15,6,1,09-Jun-13,30-Jun-14,05-Jul-14,30-Sep-15,20900.0,0,NaN,"""""""STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXXS...",NaN,01-Sep-15,01-Jul-14,NaN,NaN,NaN,NaN,NaN
1,10-Nov-15,12265,20-Oct-15,10,1,25-May-12,06-Sep-15,NaN,03-Oct-15,16201.0,10390,NaN,"""""""0000000000000000000000000000000000000000000...","""""""000000000000000000000000000XXX0000000000000...",01-Oct-15,01-Nov-12,14000.0,1400.0,NaN,3.0,5603.0
2,10-Nov-15,12265,20-Oct-15,10,1,22-Mar-12,31-Aug-15,NaN,30-Sep-15,41028.0,34420,NaN,"""""""0000000000000000000000000000000000000000000...","""""""0000000000000000000000000000000000000000000...",01-Sep-15,01-Oct-12,NaN,NaN,NaN,NaN,NaN
3,20-Jul-15,15606,09-Jul-15,10,1,13-Jan-06,NaN,26-Jul-07,31-Jan-09,93473.0,0,NaN,"""""""1200900600600600300000000000000000000000000...",NaN,01-Jul-07,01-Feb-06,NaN,NaN,NaN,NaN,NaN
4,20-Jul-15,15606,09-Jul-15,6,1,18-Jan-15,05-May-15,NaN,31-May-15,20250.0,13500,NaN,"""""""000000000000000""""""",NaN,01-May-15,01-Jan-15,NaN,NaN,NaN,NaN,NaN


In [4]:
account_train.shape

(186329, 21)

In [5]:
############ Account  #################

def account_preprocessing(df):
    lst=[]
    for data in df['paymenthistory1']:
        end_point=re.findall('...',data)
        lst.append(end_point)
    df['paymenthistory']=pd.Series(lst).values
    df[['ph1_0','ph1_1', 'ph1_2', 'ph1_3', 'ph1_4','ph1_5', 'ph1_6', 'ph1_7', 'ph1_8', 'ph1_9', 'ph1_10', 'ph1_11',
         'ph1_12', 'ph1_13','ph1_14', 'ph1_15', 'ph1_16', 'ph1_17', 
               'ph1_18','ph1_19']]=pd.DataFrame(df.paymenthistory.values.tolist())
    df['paymenthistory2']=df['paymenthistory2'].fillna('0')
    lst1=[]
    for data1 in df["paymenthistory2"]:
        end_point1=re.findall("...",data1)
        lst1.append(end_point1)
    df["paymenthistory22"]=pd.Series(lst1).values
    df[['ph2_0','ph2_1','ph2_2','ph2_3','ph2_4','ph2_5','ph2_6','ph2_7','ph2_8','ph2_9','ph2_10','ph2_11','ph2_12',
                   'ph2_13','ph2_14','ph2_15','ph2_16','ph2_17','ph2_18',
         "ph2_19"]]=pd.DataFrame(df.paymenthistory22.values.tolist())
    p_col=["paymenthistory1","paymenthistory2","paymenthistory22","paymenthistory","ph1_0","ph1_19",'ph2_0',"ph2_19"]
    df=df.drop(p_col,axis=1)
    cols=df.columns
    new_df=df[cols]
    new_df=new_df.rename(columns={'paymt_str_dt': 'paymt_end_dt', 'paymt_end_dt': 'paymt_str_dt'})
    new_df["days_between_open_close"]=(pd.to_datetime(new_df["closed_dt"],format="%d-%b-%y")-
                                             pd.to_datetime(new_df["opened_dt"],format="%d-%b-%y")).dt.days
    new_df["days_between_pend_pstart"]=(pd.to_datetime(new_df["paymt_end_dt"],format="%d-%b-%y")-
                                        pd.to_datetime(new_df["paymt_str_dt"],format="%d-%b-%y")).dt.days
    new_df["days_between_lastp_pstart"]=(pd.to_datetime(new_df["paymt_str_dt"],format="%d-%b-%y")-
                                         pd.to_datetime(new_df["last_paymt_dt"],format="%d-%b-%y")).dt.days
    new_df["days_between_lastp_acctopened"]=(pd.to_datetime(new_df["last_paymt_dt"],format="%d-%b-%y")-
                                             pd.to_datetime(new_df["opened_dt"],format="%d-%b-%y")).dt.days
    new_df["days_between_repr_pend"]=(pd.to_datetime(new_df["reporting_dt"],format="%d-%b-%y")-
                                      pd.to_datetime(new_df["paymt_end_dt"],format="%d-%b-%y")).dt.days
    new_df["credlimit_overdue"]=new_df["creditlimit"]-new_df["amt_past_due"]
    new_df["ratio_creditlimit_credit_amt"]=new_df["creditlimit"]/new_df["high_credit_amt"]
    new_df["cur_bala_actual_pay"]=new_df["actualpaymentamount"]-new_df["cur_balance_amt"]
    new_df["utilisation_trend"]=(new_df["cur_balance_amt"]/new_df["creditlimit"])
    /(new_df["cur_balance_amt"].mean()/(new_df["creditlimit"].mean()+new_df["cashlimit"]))
    new_df["ratio_curbal_creditlimit"]=new_df["cur_balance_amt"]/new_df["creditlimit"]
    return new_df

 

In [6]:
df_account_train=account_preprocessing(account_train)

# Data Preprocessing


In [7]:
############# data #############

def data_preprocessing(df):
    df=df.dropna(thresh=(0.5*len(df)),axis=1)
    df['feature_39']=pd.to_datetime(df['feature_39'].fillna(0).astype('int'))
    df['work_position_account_opened']=(pd.to_datetime(df['dt_opened'],format='%d-%b-%y')-
                                                (pd.to_datetime(df['feature_39'],format='%Y'))).dt.days/365
    df['age']=((pd.to_datetime(df['dt_opened'],format='%d-%b-%y')-pd.to_datetime(df['feature_21'],
                    format='%d-%b-%y')).dt.days/365).abs()
    df['age']=df['age'].fillna(np.mean(df['age']))
    df['saving_account_year_open']=(pd.to_datetime(df['dt_opened'],format='%d-%b-%y')-
                                            (pd.to_datetime(df['feature_53'],format='%d-%b-%y'))).dt.days/365
    df['activation']=(pd.to_datetime(df['dt_opened'],format='%d-%b-%y')-
                              (pd.to_datetime(df['entry_time'],format='%d-%b-%y'))).dt.days/365
    un_col=["feature_24","feature_28","feature_43","feature_1","dt_opened","entry_time",
          "feature_2","feature_50","feature_5","feature_6","feature_54","feature_21",
          "feature_30","feature_39","feature_53",
          "feature_22","feature_77","feature_20","feature_47","feature_70",
          "feature_66","feature_69","feature_56","feature_64","feature_63","feature_75"]
    df=df.drop(un_col,axis=1)
    df=df.rename(columns ={'feature_27': 'education','feature_12':"card_type","feature_3":"card_charge",
                         "feature_7":"card_limit","feature_32":"residence_type","feature_46":"proof_type",
                        "feature_51":"Bank_name","feature_35":"salary","feature_38":"designation"})
    df=df.reset_index()
    return df
 

In [8]:
df_data_train=data_preprocessing(data_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [9]:
def enq_table(df):
    #create an enquiry table
    
    table=pd.pivot_table(df,index=['customer_no','enq_purpose'],aggfunc='count')
    table=table[['dt_opened']]
    table=table.rename(columns={'dt_opened':'count'})
    
    t_amount=pd.pivot_table(df,index=['customer_no','enq_purpose'],aggfunc='mean')
    
    t_enq=pd.merge(t_amount,table,left_index=True,right_index=True)
    t_enq_groupby=pd.DataFrame(t_enq.groupby(['customer_no'])['count'].sum())
    
    return t_enq_groupby,t_enq

In [10]:
enquiry_t,table_enq=enq_table(enquiry_train)

In [11]:
enquiry_t.head()

,count
customer_no,
1,18
2,68
3,1
4,34
5,2


In [12]:
def account_table(df):
    table_account=df[['customer_no','acct_type']]
    p_account=pd.DataFrame(table_account.groupby(['customer_no'])['acct_type'].count())
    p_account_groupby=pd.DataFrame(table_account.groupby(['customer_no'])['acct_type'].sum())
    
    return p_account_groupby

    

In [13]:
account_t=account_table(df_account_train)

In [14]:
account_t.head()

,acct_type
customer_no,
1,121
2,20
3,10
4,257
5,62


In [15]:
enquiry_ratio=pd.merge(enquiry_t,account_t,left_index=True,right_index=True)

In [16]:
enquiry_ratio.head()

,count,acct_type
customer_no,,
1,18,121
2,68,20
3,1,10
4,34,257
5,2,62


In [17]:
################ Enquiry ###################
def enquiry_preprocessing(df_enq,df_account):
    
    enqiry_t,t_enq=enq_table(df_enq)
    account_t=account_table(df_account)
    # enquiry_ratio
    enquiry_ratio=pd.merge(enqiry_t,account_t,left_index=True,right_index=True)
    enquiry_ratio["enq_ratio"]=enquiry_ratio["acct_type"]/enquiry_ratio["count"]
    enquiry_ratio=enquiry_ratio.reset_index()
    enquiry_ratio=enquiry_ratio[["customer_no","enq_ratio"]]
    
    
    df_enq["days_between_open_enquiry"]=(pd.to_datetime(df_enq["dt_opened"],format="%d-%b-%y")-
                                      pd.to_datetime(df_enq["enquiry_dt"],format="%d-%b-%y")).dt.days
    df_days=pd.DataFrame(df_enq.groupby(["customer_no","enq_purpose"])["days_between_open_enquiry"].mean())
    
    df_days.columns=["mean_enquiry_days"]
    t_enq=t_enq[["enq_amt"]]
    
    #merging
    
    final_enq=t_enq.merge(df_days,left_index=True,right_index=True)
    final_enq=final_enq.reset_index()

    final_enq.columns=["customer_no","acct_type","mean_enq_amt","mean_enquiry_days"]
    final_enq["acct_type"]=final_enq["acct_type"].astype("int")
    print(final_enq.shape)
    
    return final_enq,enquiry_ratio


In [18]:
df_enquiry_train,df_enquiry_ratio=enquiry_preprocessing(enquiry_train,df_account_train)

(78952, 4)


In [19]:
df_enquiry_train.head()

,customer_no,acct_type,mean_enq_amt,mean_enquiry_days
0,1,0,5.000000e+04,1361.000000
1,1,2,3.500000e+06,120.000000
2,1,5,5.000000e+05,409.000000
3,1,10,6.293929e+04,1623.714286
4,2,0,3.312767e+07,738.142857


In [20]:
df_enquiry_train.shape

(78952, 4)

# Combining all files

In [21]:
data_enq_account_train=df_account_train.merge(df_enquiry_train,on=['customer_no','acct_type'],how='outer'
                                          ).merge(df_enquiry_ratio,on='customer_no').merge(df_data_train,on='customer_no')

In [22]:
list(data_enq_account_train.columns)

['dt_opened',
 'customer_no',
 'upload_dt',
 'acct_type',
 'owner_indic',
 'opened_dt',
 'last_paymt_dt',
 'closed_dt',
 'reporting_dt',
 'high_credit_amt',
 'cur_balance_amt',
 'amt_past_due',
 'paymt_end_dt',
 'paymt_str_dt',
 'creditlimit',
 'cashlimit',
 'rateofinterest',
 'paymentfrequency',
 'actualpaymentamount',
 'ph1_1',
 'ph1_2',
 'ph1_3',
 'ph1_4',
 'ph1_5',
 'ph1_6',
 'ph1_7',
 'ph1_8',
 'ph1_9',
 'ph1_10',
 'ph1_11',
 'ph1_12',
 'ph1_13',
 'ph1_14',
 'ph1_15',
 'ph1_16',
 'ph1_17',
 'ph1_18',
 'ph2_1',
 'ph2_2',
 'ph2_3',
 'ph2_4',
 'ph2_5',
 'ph2_6',
 'ph2_7',
 'ph2_8',
 'ph2_9',
 'ph2_10',
 'ph2_11',
 'ph2_12',
 'ph2_13',
 'ph2_14',
 'ph2_15',
 'ph2_16',
 'ph2_17',
 'ph2_18',
 'days_between_open_close',
 'days_between_pend_pstart',
 'days_between_lastp_pstart',
 'days_between_lastp_acctopened',
 'days_between_repr_pend',
 'credlimit_overdue',
 'ratio_creditlimit_credit_amt',
 'cur_bala_actual_pay',
 'utilisation_trend',
 'ratio_curbal_creditlimit',
 'mean_enq_amt',
 'mea

In [23]:
# Creating train data
def input_and_target(df):
    d_col=['dt_opened','upload_dt','opened_dt','last_paymt_dt','closed_dt','reporting_dt','paymt_end_dt','paymt_str_dt']
    t_data=df.drop(d_col,axis=1)
    t_data['Target']=t_data['Bad_label'].astype('category')
    t_data=t_data.drop('Bad_label',axis=1)
    return t_data
    

In [24]:
train=input_and_target(data_enq_account_train)

In [25]:
train.columns

Index(['customer_no', 'acct_type', 'owner_indic', 'high_credit_amt',
       'cur_balance_amt', 'amt_past_due', 'creditlimit', 'cashlimit',
       'rateofinterest', 'paymentfrequency',
       ...
       'feature_71', 'feature_72', 'feature_76', 'feature_78', 'feature_79',
       'work_position_account_opened', 'age', 'saving_account_year_open',
       'activation', 'Target'],
      dtype='object', length=107)

# Feature Engineering

In [26]:
def selcols(prefix,a=1,b=18):
    return [prefix+str(i) for i in np.arange(a,b+1)]


In [27]:
def feature_engineering(df):
    df['day_past_due_avg_1']=df[selcols('ph1_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'None':0},
                                                                 regex=True).dropna().astype(int).mean(axis=1)
    df['day_past_due_std_1'] = df[selcols('ph1_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'None':0},
                                                                    regex=True).dropna().astype(int).std(axis=1)
    df['day_past_due_avg_2'] = df[selcols('ph2_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'None':0},
                                                                    regex=True).dropna().astype(int).mean(axis=1)
    df['day_past_due_std_2'] = df[selcols('ph2_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'None':0},
                                                                    regex=True).dropna().astype(int).std(axis=1)
    
    df['past_due_months_avg'] = (df['day_past_due_avg_1']+df['day_past_due_avg_2'])/30
    df['past_due_months_std'] = (df['day_past_due_std_1']+df['day_past_due_std_2'])/30
    
    df['bill_amt']=df['cur_balance_amt']+df['actualpaymentamount']
    df['bill_relamt']=df['bill_amt']/df['creditlimit']
    
    
    #### Feature Scaling ######
    df[selcols('ph1_')]= df[selcols('ph1_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'None':0},
                                                             regex=True).dropna().astype(int)
    df[selcols('ph2_')]= df[selcols('ph2_')].replace({'[A-Z]':0,'"""':0,'XXX':0,'Non':0},
                                                             regex=True).dropna().astype(int)
    return df
    
    

In [28]:
df_train=feature_engineering(train)

In [29]:
df_train.to_csv('D:/Projects/Tookitaki/train.csv',index=False)

# Label Encoding

In [30]:
def label_encoding(df):
    df_label=df.select_dtypes(include=['object','category'])
    df_float=df_train.select_dtypes(exclude=['object','category'])
    
    le=LabelEncoder()
    for each in df_label.columns.values:
        if df_label[each].dtypes=='object':
            le.fit(df_label[each].astype(str))
            df_label[each]=le.transform(df_label[each].astype(str))
    
    
    
    scaler=StandardScaler()
    df_float=df_float.fillna(-1)
    col_names=df_float.columns
    df_float=pd.DataFrame(scaler.fit_transform(df_float))
    df_float.columns=col_names
    
    y_train=df_label['Target']
    x_train=df_label.join(df_float).drop('Target',axis=1)
    
    return x_train,y_train
    
    

In [31]:
x_train,y_train=label_encoding(df_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# TEST DATA

In [32]:
##### data preprocessing #####
df_data_test=data_preprocessing(data_test)
df_account_test=account_preprocessing(account_test)
df_enquiry_test,df_enquiry_ratio=enquiry_preprocessing(enquiry_test,df_account_test)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

(33765, 4)


In [33]:
account_test.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount,paymenthistory,ph1_0,ph1_1,ph1_2,ph1_3,ph1_4,ph1_5,ph1_6,ph1_7,ph1_8,ph1_9,ph1_10,ph1_11,ph1_12,ph1_13,ph1_14,ph1_15,ph1_16,ph1_17,ph1_18,ph1_19,paymenthistory22,ph2_0,ph2_1,ph2_2,ph2_3,ph2_4,ph2_5,ph2_6,ph2_7,ph2_8,ph2_9,ph2_10,ph2_11,ph2_12,ph2_13,ph2_14,ph2_15,ph2_16,ph2_17,ph2_18,ph2_19
0,01-Oct-15,8226,07-Sep-15,10,1,23-Jul-15,NaN,NaN,20-Aug-15,31937.0,31937,NaN,"""""""000""""""",0,01-Aug-15,01-Aug-15,54000.0,10000.0,39,3.0,NaN,"["""""", 000, """"""]","""""""",000,"""""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,01-Oct-15,8226,07-Sep-15,10,1,27-Mar-15,09-Aug-15,NaN,22-Aug-15,51311.0,34683,NaN,"""""""000000000000000""""""",0,01-Aug-15,01-Apr-15,60000.0,12000.0,NaN,3.0,24012.0,"["""""", 000, 000, 000, 000, 000, """"""]","""""""",000,000,000,000,000,"""""""",None,None,None,None,None,None,None,None,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,01-Oct-15,8226,07-Sep-15,5,1,28-Feb-15,07-Jul-15,NaN,31-Jul-15,150000.0,136642,NaN,"""""""000000000000000""""""",0,01-Jul-15,01-Mar-15,NaN,NaN,NaN,3.0,5274.0,"["""""", 000, 000, 000, 000, 000, """"""]","""""""",000,000,000,000,000,"""""""",None,None,None,None,None,None,None,None,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,01-Oct-15,8226,07-Sep-15,10,1,27-Aug-14,26-Jul-15,NaN,31-Jul-15,19000.0,0,NaN,"""""""000000000000000000000000000000000000""""""",0,01-Jul-15,01-Aug-14,NaN,NaN,NaN,NaN,NaN,"["""""", 000, 000, 000, 000, 000, 000, 000, 000, ...","""""""",000,000,000,000,000,000,000,000,000,000,000,000,"""""""",None,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,29-Jul-15,3439,23-Jul-15,6,1,25-Apr-15,02-Jun-15,NaN,30-Jun-15,18500.0,10276,NaN,"""""""000000000""""""",0,01-Jun-15,01-Apr-15,NaN,NaN,NaN,NaN,NaN,"["""""", 000, 000, 000, """"""]","""""""",000,000,000,"""""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [34]:
#########   Combining all files   ##########
data_enq_account_test=df_account_test.merge(df_enquiry_test,on=['customer_no','acct_type'],how='outer'
                                          ).merge(df_enquiry_ratio,on='customer_no').merge(df_data_test,on='customer_no')


In [35]:
list(data_enq_account_test)

['dt_opened',
 'customer_no',
 'upload_dt',
 'acct_type',
 'owner_indic',
 'opened_dt',
 'last_paymt_dt',
 'closed_dt',
 'reporting_dt',
 'high_credit_amt',
 'cur_balance_amt',
 'amt_past_due',
 'paymt_end_dt',
 'paymt_str_dt',
 'creditlimit',
 'cashlimit',
 'rateofinterest',
 'paymentfrequency',
 'actualpaymentamount',
 'ph1_1',
 'ph1_2',
 'ph1_3',
 'ph1_4',
 'ph1_5',
 'ph1_6',
 'ph1_7',
 'ph1_8',
 'ph1_9',
 'ph1_10',
 'ph1_11',
 'ph1_12',
 'ph1_13',
 'ph1_14',
 'ph1_15',
 'ph1_16',
 'ph1_17',
 'ph1_18',
 'ph2_1',
 'ph2_2',
 'ph2_3',
 'ph2_4',
 'ph2_5',
 'ph2_6',
 'ph2_7',
 'ph2_8',
 'ph2_9',
 'ph2_10',
 'ph2_11',
 'ph2_12',
 'ph2_13',
 'ph2_14',
 'ph2_15',
 'ph2_16',
 'ph2_17',
 'ph2_18',
 'days_between_open_close',
 'days_between_pend_pstart',
 'days_between_lastp_pstart',
 'days_between_lastp_acctopened',
 'days_between_repr_pend',
 'credlimit_overdue',
 'ratio_creditlimit_credit_amt',
 'cur_bala_actual_pay',
 'utilisation_trend',
 'ratio_curbal_creditlimit',
 'mean_enq_amt',
 'mea

In [36]:
######## creating test data   #######
test=input_and_target(data_enq_account_test)

In [37]:
test.columns

Index(['customer_no', 'acct_type', 'owner_indic', 'high_credit_amt',
       'cur_balance_amt', 'amt_past_due', 'creditlimit', 'cashlimit',
       'rateofinterest', 'paymentfrequency',
       ...
       'feature_71', 'feature_72', 'feature_76', 'feature_78', 'feature_79',
       'work_position_account_opened', 'age', 'saving_account_year_open',
       'activation', 'Target'],
      dtype='object', length=107)

In [38]:
######### Feature engineering ######## 
df_test=feature_engineering(test)

In [39]:
df_test.to_csv('D:/Projects/Tookitaki/test.csv',index=False)

In [40]:
###### Label encoding #########
x_test,y_test=label_encoding(df_test)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [41]:
x_train.shape,y_train.shape

((209864, 114), (209864,))

In [45]:
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: category
Categories (2, int64): [0, 1]

In [50]:
type(y_train)

pandas.core.series.Series

# Models

In [46]:
# 1. Logistic Regression

def logistic_regression(x_train,y_train,x_test,y_test):
        
    model=linear_model.LogisticRegression()
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy

    
    
#  2.  Random Forest

def random_forest(x_train,y_train,x_test,y_test):
    
    model=RandomForestClassifier(n_estimators=500,min_samples_leaf=5)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy




#  3.   KNN Classifier

def knn_classifier(x_train,y_train,x_test,y_test):
    
    model=KNeighborsClassifier()
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy


#  4. SVM

def svm_classifier(x_train,y_train,x_test,y_test):
    
    model=SVC()
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy


#  5. Naive Bayes

def naiv_bayes(x_train,y_train,x_test,y_test):
    
    model=GaussianNB()
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy


#  6. Xgboost

def xgb_boosting(x_train,y_train,x_test,y_test):
    
    model=xgb.XGBClassifier()
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=np.mean(y_pred==y_test)
        
    return accuracy
    
    

        

In [47]:
xgb_accuracy=xgb_boosting(x_train,y_train,x_test,y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
xgb_accuracy

0.9599088482032149

In [51]:
nb_accuracy=naiv_bayes(x_train,y_train,x_test,y_test)

In [52]:
nb_accuracy


0.9318260519878018

In [53]:
lrc_accuracy=logistic_regression(x_train,y_train,x_test,y_test)


In [54]:
lrc_accuracy

0.9601992828498341

# Generate output

So I'm choosing logistic regression model for the geration of output